In [1]:
#Setup
import pandas as pd
import numpy as np

In [2]:
#Reading Data
isi_auftrag = pd.read_csv("../data/ISI_Daten/ISI_Auftragsdetails_s822.csv", sep="|")
isi_aufenthalt = pd.read_csv("../data/ISI_Daten/ISI_FzAufenthalte.csv", sep="|")
isi_stamm = pd.read_csv("../data/ISI_Daten/ISI_Stammdaten.csv", sep="|")

In [3]:
isi_aufenthalt.tail()

,FZ-ID,WHO_NUMMER,WERK,BAUART,FZ_TYP,HZG,ILA_GRUPPE,REGION,DAT_WERKSBEZUG,DAT_EINBUCH,PLAN_ÜBERGABE_1,PLAN_ÜBERGABE_2,DAT_AUSBUCH
70514,1f63477c-c3ca-48b2-ad0c-7f608bfa5601,1000000227,RBB Werkstatt Göttingen,O2A55,NaN,AIH,Bedarf,Nord,43503,43503,43504.0,43504.0,43504.0
70515,5ace5176-4427-4de4-a01a-38ce4040c86a,1000000045,Werk NL Ulm,O2C50,NaN,AIH,Bedarf,Schiene,43487,43487,43487.0,43487.0,43487.0
70516,5d441b19-7e98-4580-ad27-c79fff15d535,1000000135,Werk Betrieb Osnabrück,O2B15,NaN,AIH,Bedarf,Nord,43487,43487,43487.0,43487.0,43494.0
70517,7a48e4a3-5f11-47a6-9b69-e3c8d82cb141,1000000101,Werkstatt Oberhausen,O4C50,NaN,AIH,Bedarf,NRW,43634,43634,43634.0,43634.0,43634.0
70518,3d233fd5-3005-4aac-9fb6-9cff3da5f80a,1000000097,Werkstatt Oberhausen,O2I50,NaN,AIH,Bedarf,NRW,43699,43699,43699.0,43699.0,43699.0


# Auftragsdetails

In [4]:
#Duplikate entfernen
isi_auftrag = isi_auftrag.drop_duplicates()

#Spaltennamen anpassen
isi_auftrag = isi_auftrag.rename(columns={'FZ_NR': 'FZ-ID'})

#Unnötige Spalten löschen
isi_auftrag = isi_auftrag.drop(['WERK'], axis=1)

# Aufenthalt

In [5]:
#Duplikate entfernen
isi_aufenthalt = isi_aufenthalt.drop_duplicates()

#WHO-Nummer umrechnen
isi_aufenthalt['WHO_NUMMER'] = isi_aufenthalt['WHO_NUMMER'] - 1000000000

#Unnötige Spalten entfernen
isi_aufenthalt = isi_aufenthalt.drop(['FZ_TYP', 'HZG', 'DAT_EINBUCH', 'PLAN_ÜBERGABE_1', 'PLAN_ÜBERGABE_2'], axis=1)

#Timestamp "DAT_AUSBUCH" zu Integer wechseln
isi_aufenthalt["DAT_AUSBUCH"] = isi_aufenthalt.apply(
    lambda row: 0 if np.isnan(row['DAT_AUSBUCH']) else row['DAT_AUSBUCH'],
    axis=1)

isi_aufenthalt["DAT_AUSBUCH"] = isi_aufenthalt["DAT_AUSBUCH"].astype(int)

# Stammdaten

In [6]:
#Duplikate entfernen
isi_stamm = isi_stamm.drop_duplicates()

#Busmaße in Meter umwandeln
isi_stamm['FZ_LAENGE'] = isi_stamm['FZ_LAENGE'] / 1000
isi_stamm['FZ_BREITE'] = isi_stamm['FZ_BREITE'] / 1000
isi_stamm['FZ_HOEHE'] = isi_stamm['FZ_HOEHE'] / 1000

#Unnötige Spalten löschen
isi_stamm = isi_stamm.drop(['WERKSTATT_NAME', 'CHASSIS_BAUART', 'D2_TYP', 'KRAFTSTOFF_ART', 'NENNLEISTUNG', 'MARKE', 'ZULASSUNGSDATUM', 'REISEGESCHWINDGK'], axis=1)

#Plätze kombinieren und alte Spalten löschen
isi_stamm['PLAETZE'] = isi_stamm['SITZPLAETZE'] + isi_stamm['STEHPLAETZE']
isi_stamm = isi_stamm.drop(['SITZPLAETZE','STEHPLAETZE' ], axis=1)

#Inaktive Fahrzeuge entfernen
isi_stamm = isi_stamm[isi_stamm['AKTIV'] != 0]

#Klima kategorisieren
isi_stamm['KLIMAANLAGE'] = isi_stamm['KLIMAANLAGE'].astype('category').cat.codes
isi_stamm['KLIMAANLAGE'] = isi_stamm['KLIMAANLAGE'] + 1

#Anpassungen
isi_stamm = isi_stamm.rename(columns={'BABR': 'BAUART'})

# Clean Aufträge

In [7]:
isi_auftraege = pd.read_csv("../data/ISI_Daten/ISI_Auftraege.csv", sep="|")

/Users/alexmayr/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
#Duplikate entfernen
isi_auftraege = isi_auftraege.drop_duplicates()

#Unnötige Spalten löschen
isi_auftraege = isi_auftraege.drop(['BAHNSTELLE', 'ARBEITSPLATZ', 'ILA', 'AUSGANG'], axis=1)

#WHO-Nummer umrechnen
isi_auftraege['WHO_NUMMER'] = isi_auftraege['WHO_NUMMER'] - 1000000000


#Anpassungen
isi_auftraege = isi_auftraege.rename(columns={'FZ_NR': 'FZ-ID'})

In [9]:
isi_auftraege.head(1)

,FZ-ID,WERK,AUFTRAG,AUFTRAGSTEXT,WHO_NUMMER,AUFTRAGSART,SYSTEMSTATUS,ECKSTARTTERMIN,ECKENDTERMIN,BEZUGSDATUM,UPDATE_DATE,UEBERGEORDN_AUFTRAG
0,1d707286-e40a-4d8d-8c5d-47cff0169954,NaN,1032929649,Inst.schw.Unfallsch./G3.9 (Gw),158,610M,ABGS RÜCK ABRV MABS TDRU VOKL WABE,43102,43102,43144,43510.536806,NaN


# Angleichung der jewweils vorhandenen FZ-IDS

In [24]:
isi_stamm = isi_stamm[isi_stamm['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

In [25]:
isi_auftraege = isi_auftraege[isi_auftraege['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

In [26]:
isi_aufenthalt = isi_aufenthalt[isi_aufenthalt['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

In [27]:
isi_aufrag = isi_aufenthalt[isi_aufenthalt['FZ-ID'].isin(isi_aufenthalt['FZ-ID'])]

In [28]:
isi_stamm = isi_stamm[isi_stamm['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

In [29]:
isi_auftraege = isi_auftraege[isi_auftraege['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

In [30]:
isi_aufenthalt = isi_aufenthalt[isi_aufenthalt['FZ-ID'].isin(isi_auftrag['FZ-ID'])]

In [31]:
isi_aufenthalt['FZ-ID'].nunique()

5519

In [32]:
isi_auftrag['FZ-ID'].nunique()

5668

In [36]:
isi_auftrag.tail()

,FZ-ID,BAUART,ISAS,AUFTRAGSNR,AUFTRAGSBEZEICHNUNG,PRODUKTCODE,PRODUKTBEZEICHNUNG,MONAT,JAHR
368485,6c627e63-5273-497a-a599-14bc8d8a0307,O2B53,030,1035883280,außerplanm. Instandsetzung,O00008500AB02,Motor und Motorraum reinigen,7,2019
368486,fcf187b9-7c1d-4c04-8584-4b2e2bf306ff,O2C50,030,1035152868,außerplanm. Instandsetzung,O42000422CD02,Bremsnachsteller 2. Achse nB ern.,3,2019
368487,fcf187b9-7c1d-4c04-8584-4b2e2bf306ff,O2C50,030,1035152868,außerplanm. Instandsetzung,O42000586CD02,Bremsscheiben HA prf. nB. ern.,3,2019
368488,fcf187b9-7c1d-4c04-8584-4b2e2bf306ff,O2C50,030,1035152868,außerplanm. Instandsetzung,O46000001CD02,Schaden Lenkung Langtext prüf.,3,2019
368489,fcf187b9-7c1d-4c04-8584-4b2e2bf306ff,O2C50,030,1035152868,außerplanm. Instandsetzung,O54002000CD02,Relais prf.nB.ern.Ortsangabe:,3,2019


# IDs Filtern (?)

In [43]:
ids_auftrag = isi_auftrag['FZ-ID'].unique()
ids_aufenthalt = isi_aufenthalt['FZ-ID'].unique()
ids_stamm = isi_stamm['FZ-ID'].unique()


common_ids = list(set(ids_auftrag) & set(ids_aufenthalt) & set(ids_stamm))
common_ids =list(map(str, common_ids))


isi_auftrag = isi_auftrag[~isi_auftrag['FZ-ID'].isin(common_ids)]
isi_aufenthalt = isi_aufenthalt[~isi_aufenthalt['FZ-ID'].isin(common_ids)]
isi_stamm = isi_stamm[~isi_stamm['FZ-ID'].isin(common_ids)]


In [44]:
pd.DataFrame(common_ids).to_csv("../Cleaned_Data/ids.csv", index = "FZ-ID")

In [51]:
ids = pd.read_csv("../Cleaned_Data/ids.csv", sep=",")

In [52]:
ids = ids.rename(columns={'0': 'FZ-ID'})

# Neue Daten speichern

In [54]:
isi_stamm.to_csv("../Cleaned_Data/isi_stammdaten.csv", index = False)
isi_aufenthalt.to_csv("../Cleaned_Data/isi_aufenthalt.csv", index = False)
isi_auftrag.to_csv("../Cleaned_Data/isi_auftrag.csv", index = False)
isi_auftraege.to_csv("../Cleaned_Data/isi_auftraege.csv", index = False)

In [55]:
ids.to_csv("../Cleaned_Data/ids.csv", index = False)